# Imports

In [ ]:
## Import necessary libraries here
import os
import random
import cv2
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
from google.colab.patches import cv2_imshow
import itertools
import copy
from math import sqrt
import math
from scipy.linalg import sqrtm
import plotly.graph_objects as go

# Part 1: Epipolar Geometry


## Overview

In this problem, you will implement an algorithm for automatically estimating homography with RANSAC. In the file matches.mat, we provide the detected Harris corners row-column positions in variables r1 c1 for the first image; variables r2 c2 for the second image; and the corresponding matched pairs in the variable matches.

<!-- <img src="https://drive.google.com/uc?id=1Tr723u5OXmwkd4RDmu9z886ITJU9j1cL&export=download" width="800"/> -->

<img src="https://drive.google.com/uc?id=1cCT_YU4xp-jRqh1dfoW5JE6MIZH-DDwP" width="800"/>

The outline of the normalized 8-point algorithm:

<img src="https://drive.google.com/uc?id=1rw0Y7XOfqKhcNj2boJIWKJYje2a7mHGd" width="700"/>

### Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 1cn3_SscjlLrf4BzUWe8MV-XqMqBY4Nj_
!unzip "/content/Part1_data.zip" -d "/content/"
# Load Matches
data = loadmat('/content/Part1_data/matches.mat')
r1 = data['r1']
r2 = data['r2']
c1 = data['c1']
c2 = data['c2']
matches = data['matches']

Downloading...
From: https://drive.google.com/uc?id=1cn3_SscjlLrf4BzUWe8MV-XqMqBY4Nj_
To: /content/Part1_data.zip

  0% 0.00/157k [00:00<?, ?B/s]
100% 157k/157k [00:00<00:00, 77.2MB/s]
Archive:  /content/Part1_data.zip
   creating: /content/Part1_data/
  inflating: /content/Part1_data/chapel00.png  
  inflating: /content/Part1_data/chapel01.png  
  inflating: /content/Part1_data/matches.mat  


In [ ]:
# Load Keypoints
x1 = c1[matches[:,0]-1]
y1 = r1[matches[:,0]-1]
x2 = c2[matches[:,1]-1]
y2 = r2[matches[:,1]-1]

### Code

In [ ]:
def normalize(x, y):
  # Function: find the transformation to make it zero mean and the variance as sqrt(2)
  std_x = np.std(x)
  std_y = np.std(y)
  mean_x = np.mean(x)
  mean_y = np.mean(y)
  T = np.array([[1/std_x,     0  , -mean_x/std_x],
                [    0  , 1/std_y, -mean_y/std_y],
                [    0  ,     0  ,      1       ]])
  normalized_coords = []
  for i in range(x.shape[0]):
    temp = np.matmul(T, np.array([x[i], y[i], 1], dtype=object))
    normalized_coords.append([temp[0][0], temp[1][0], temp[2][0]])
  normalized_coords = np.array(normalized_coords)
  normalized_x = normalized_coords[:, 0]
  normalized_y = normalized_coords[:, 1]
  return normalized_x, normalized_y, T
  
def ransacF(x1, y1, x2, y2, do_normalization=True, T1=None, T2=None):
  if do_normalization:
    # Find normalization matrix & Transform point set 1 and 2
    x1, y1, T1 = normalize(x1, y1)
    x2, y2, T2 = normalize(x2, y2)
  # RANSAC based 8-point algorithm
  threshold = 0.1
  max_num_inliers = -1
  num_iters = 0
  rand_indices = [i for i in range(x1.shape[0])]
  random.shuffle(rand_indices)
  for subset in itertools.combinations(rand_indices, 8):
    num_iters += 1
    if num_iters>1400:
      break
    x1_subset = x1[tuple([subset])]
    y1_subset = y1[tuple([subset])]
    x2_subset = x2[tuple([subset])]
    y2_subset = y2[tuple([subset])]
    F = computeF(x1_subset,y1_subset,x2_subset,y2_subset)
    num_inliers, distances = getInliers(x1, y1, x2, y2, F, threshold)
    if num_inliers > max_num_inliers:
      inlier_indices = tuple(np.arange(x1.shape[0])[distances==-1])
      max_num_inliers = num_inliers
      max_inlier_comb = inlier_indices 
  
  print('Number of iterations =', num_iters)
  print('Number of inliers =', max_num_inliers)
  x1_subset = x1[tuple([max_inlier_comb])]
  y1_subset = y1[tuple([max_inlier_comb])]
  x2_subset = x2[tuple([max_inlier_comb])]
  y2_subset = y2[tuple([max_inlier_comb])]
  F_norm = computeF(x1_subset,y1_subset,x2_subset,y2_subset)
  F = np.matmul(T1.T, np.matmul(F_norm, T2))
  return F, max_inlier_comb

def getInliers(x1, y1, x2, y2, F, threshold):
  # Function: implement the criteria checking inliers.
  distances = []
  for j in range(x1.shape[0]):
    line = np.matmul(F, np.array([x2[j], y2[j], 1]))
    temp = abs(np.matmul(np.array([x1[j], y1[j], 1]), line)) / sqrt(line[0]**2 + line[1]**2)
    distances.append(temp)
  distances = np.array(distances)
  assert np.amin(distances) >= 0, str(np.amin(distances))+' is the min number'
  distances[distances<=threshold] = -1
  distances[distances>threshold] = 0
  num_inliers = np.sum(distances) / -1
  return num_inliers, distances

  
def computeF(x1, y1, x2, y2):
  #  Function: compute fundamental matrix from corresponding points
  A = []
  for i in range(x1.shape[0]):
    A.append([x1[i]*x2[i], x1[i]*y2[i], x1[i], y1[i]*x2[i], y1[i]*y2[i], y1[i], x2[i], y2[i], 1])
  U, S, VT = np.linalg.svd(A)
  f = VT[-1]
  F = np.reshape(f, (3,3))
  u, s, vt = np.linalg.svd(F)
  s[-1] = 0
  s = np.diag(s)
  F = np.matmul(u, np.matmul(s, vt))
  return F

def gen_epi_line_and_plot(epi_line, x, y, img):
  rows, cols, _ = img.shape
  start1 = (0, int(-epi_line[2][0]/epi_line[1][0]))
  x_ = cols-1
  y_ = int(((-epi_line[0][0]*x_)-epi_line[2][0]) / epi_line[1][0])
  end1 = (x_, y_)
  img = cv2.line(img, start1, end1, (0, 255, 0), 1)
  img = cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1)
  return img

def plot_epi_lines(epi_line_1, epi_line_2, x_1, y_1, x_2, y_2, im1, im2, show_result):
  im1 = gen_epi_line_and_plot(epi_line_1, x_1, y_1, im1)
  im2 = gen_epi_line_and_plot(epi_line_2, x_2, y_2, im2)
  if show_result:
    combined_img = np.hstack((im1, im2))
    cv2_imshow(combined_img)

img1 = cv2.imread('/content/Part1_data/chapel00.png')
img2 = cv2.imread('/content/Part1_data/chapel01.png')
F, max_inlier_comb = ransacF(x1, y1, x2, y2)
F_unit = F / np.sqrt(np.sum(np.square(F)))
print(F_unit)
rand_7_indices=[]
while np.unique(rand_7_indices).shape[0] != 7:
  rand_7_indices = (np.random.rand(7)*x1.shape[0]).astype(int)

for rand_idx in rand_7_indices:
  x_1 = x1[rand_idx]
  y_1 = y1[rand_idx]
  x_2 = x2[rand_idx]
  y_2 = y2[rand_idx]
  epi_line_1 = np.matmul(F, np.array([x_2, y_2, 1], dtype=object))
  epi_line_2 = np.matmul(F.T, np.array([x_1, y_1, 1], dtype=object))
  plot_epi_lines(epi_line_1, epi_line_2, x_1, y_1, x_2, y_2, img1, img2, rand_idx==rand_7_indices[-1])

### Write-up
<!-- *   Describe what test you used for deciding inlier vs. outlier.
*   Display the estimated fundamental matrix F after normalizing to unit length
*   Randomly select 7 sets of matching points. Plot the corresponding epipolar lines and the points on each image. Show the two images (with plotted points and lines) next to each other. -->

<!-- *   Plot the outlier keypoints with green dots on top of the first image -->
<!-- *   Randomly select 7 sets of matching points. Plot the corresponding epipolar lines ('g’) and the points (with 'r+’) on each image. Show the two images (with plotted points and lines) next to each other. -->



- 1) Test used for deciding inlier vs. outlier.
  - Given a pair of matches x1 & x2, I computed the epipolar line of x2 in image1 & found the distance of the point x1 from the above computed epipolar line in img1 and thresholded it to decide if the pair of matches is an outlier.

- 2) Fundamental Matrix (after normalizing to unit length):

  <img src="https://drive.google.com/uc?id=1OdJPrN4R5N9Q_Kdk-OBtv_cAnbt_EkKb" width="400"/>

- 3) Epipolar Lines & associated points:

  <img src="https://drive.google.com/uc?id=1BLWXQE0PlyWIvBNsqK_qhNRYkK9h7YSq" width="800"/>